In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import pandas as pd

training_data = datasets.FashionMNIST(root="data", train=True,
    download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False,
    download=True, transform=ToTensor())

torch.manual_seed(23)
train_dataloader = DataLoader(training_data, batch_size=64,
  shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64,
  shuffle=True)

class NeuralNetwork(nn.Module):
  def __init__(self, input_size, layers: list, num_classes):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.layers = nn.ModuleList()
    self.input_size = input_size
    for output_size, activation_function in layers:
      self.layers.append(nn.Linear(input_size, output_size))
      input_size = output_size
      self.layers.append(activation_function)
    self.layers.append(nn.Linear(input_size, num_classes))
  def forward(self, x):
    x = self.flatten(x)
    for layer in self.layers:
      x = layer(x)
    return x

torch.manual_seed(23)
C = 10
N,H,W = training_data.data.shape; D = H*W

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X); loss = loss_fn(pred, y)
        optimizer.zero_grad(); loss.backward(); optimizer.step() # backprop
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"trloss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset); nbatches = len(dataloader)
    teloss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X); teloss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    teloss /= nbatches; correct /= size
    # print(f"teacc: {(100*correct):>0.1f}%, teloss: {teloss:>8f} \n")
    teacc = 100*correct
    return teacc, teloss

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
from google.colab import files

In [ ]:
### Experimental parameters
new_M = 512
new_epochs = 10
new_learning_rate = 1e-3
###

def experiment(new_epochs, new_M, new_learning_rate):
  M1, M2 = new_M, new_M # 512, 512 # Main adjustable parameter [256, 512, 1024]
  model = NeuralNetwork(D, [(M1, nn.ReLU()), (M2, nn.ReLU())], C)

  learning_rate = new_learning_rate # 1e-3 # Main adjustable parameter [1e-4, 1e-3, 1e-2]
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  epochs = new_epochs # 10 # Main adjustable parameter [20]

  data = pd.DataFrame(columns=['teloss', 'teacc', 'epoch', 'num_neurons', 'learning_rate'])

  for t in range(epochs):
      print(f"Epoch {t+1}, num_neuron {new_M}, learning_rate {new_learning_rate}")
      train_loop(train_dataloader, model, loss_fn, optimizer)
      teacc, teloss = test_loop(test_dataloader, model, loss_fn)
      data.loc[len(data)] = [teloss, teacc, t+1, new_M, new_learning_rate]

  dataFile = f'num_neurons_{new_M}-learning_rate_{new_learning_rate}'
  data.to_csv(f'/content/results/{dataFile}.csv', index=False)
  print(f"Results successfully saved to the file '/content/results/{dataFile}.csv'.")

In [ ]:
epochs = 20
for num_neuron in [256, 512, 1024, 2048]:
  for learning_rate in [1e-5, 1e-4, 1e-3, 1e-2]:
    experiment(epochs, num_neuron, learning_rate)

In [ ]:
epochs = 20
for num_neuron in [2048]:
  for learning_rate in [1e-4, 1e-3, 1e-2]:
    experiment(epochs, num_neuron, learning_rate)

Epoch 1, num_neuron 2048, learning_rate 0.0001
Epoch 2, num_neuron 2048, learning_rate 0.0001
Epoch 3, num_neuron 2048, learning_rate 0.0001
Epoch 4, num_neuron 2048, learning_rate 0.0001
Epoch 5, num_neuron 2048, learning_rate 0.0001
Epoch 6, num_neuron 2048, learning_rate 0.0001
Epoch 7, num_neuron 2048, learning_rate 0.0001
Epoch 8, num_neuron 2048, learning_rate 0.0001
Epoch 9, num_neuron 2048, learning_rate 0.0001
Epoch 10, num_neuron 2048, learning_rate 0.0001
Epoch 11, num_neuron 2048, learning_rate 0.0001
Epoch 12, num_neuron 2048, learning_rate 0.0001
Epoch 13, num_neuron 2048, learning_rate 0.0001
Epoch 14, num_neuron 2048, learning_rate 0.0001
Epoch 15, num_neuron 2048, learning_rate 0.0001
Epoch 16, num_neuron 2048, learning_rate 0.0001
Epoch 17, num_neuron 2048, learning_rate 0.0001
Epoch 18, num_neuron 2048, learning_rate 0.0001
Epoch 19, num_neuron 2048, learning_rate 0.0001
Epoch 20, num_neuron 2048, learning_rate 0.0001
Results successfully saved to the file '/content/

In [ ]:
all_data = pd.DataFrame(columns=['teloss', 'teacc', 'epoch', 'num_neurons', 'learning_rate'])

epochs = 20
for num_neuron in [256, 512, 1024, 2048]:
  for learning_rate in [1e-5, 1e-4, 1e-3, 1e-2]:
    dataFile = f'num_neurons_{num_neuron}-learning_rate_{learning_rate}'
    df = pd.read_csv(f'/content/results/{dataFile}.csv')
    df = df.reset_index()
    for index, row, in df.iterrows():
      all_data.loc[len(all_data)] = [row['teloss'], row['teacc'], row['epoch'], int(row['num_neurons']), row['learning_rate']]

all_data.to_csv(f'/content/results/all_data.csv', index=False)
print(f"Results successfully saved to the file.")



Results successfully saved to the file.


In [ ]:
display(all_data)

,teloss,teacc,epoch,num_neurons,learning_rate
0,1.144522,65.54,1.0,256.0,0.00001
1,0.828813,70.31,2.0,256.0,0.00001
2,0.724212,73.66,3.0,256.0,0.00001
3,0.662784,76.45,4.0,256.0,0.00001
4,0.624259,78.02,5.0,256.0,0.00001
...,...,...,...,...,...
315,0.424950,85.33,16.0,2048.0,0.01000
316,0.435432,85.01,17.0,2048.0,0.01000
317,0.470281,84.67,18.0,2048.0,0.01000
318,0.433393,85.33,19.0,2048.0,0.01000


In [ ]:
import plotly
import plotly.graph_objs as go

res = pd.read_csv('/content/results/all_data.csv')
markercolor = all_data['num_neurons']

# Make Plotly figure
fig1 = go.Scatter3d(x=res['epoch'],
                    y=res['learning_rate'],
                    z=res['teloss'],
                    marker=dict(color=markercolor,
                                # opacity=1,
                                # reversescale=True,
                                # colorscale='Blues',
                                size=3),
                    line=dict (width=0.02),
                    mode='markers')


#Make Plot.ly Layout
mylayout = go.Layout(scene=dict(xaxis=dict( title="epoch"),
                                yaxis=dict( title="learning_rate",type="log"),
                                zaxis=dict(title="teloss")),)

#Plot and save html
plotly.offline.plot({"data": [fig1],
                     "layout": mylayout},
                     auto_open=True,
                     filename=("4DPlot.html"))



NameError: ignored

In [ ]:
import plotly.express as px
df = pd.read_csv('/content/results/all_data.csv')
df["num_neurons"] = df["num_neurons"].astype(str)
fig = px.scatter_3d(df, x='epoch', y='learning_rate', z='teloss',
              color='num_neurons', log_y=True)

fig.update_traces(marker=dict(size=3))

fig.show()
fig.write_html("all_data_plot.html")

In [ ]:
best_row = df.loc[df['teloss'].idxmin()]
display(best_row)

teloss           0.300772
teacc               89.87
epoch                13.0
num_neurons        2048.0
learning_rate      0.0001
Name: 272, dtype: object